In [1]:
import pandas as pd
from pyspark.sql import *

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlc = SQLContext(sc)

muertes = sqlc.read.csv("muertes.csv",header=True).rdd
muertes = muertes.map(lambda x:(x[0],pd.to_datetime(x[1],"coerce"),x[2],x[3],x[4])).cache()
proporciones_razas = sqlc.read.csv("proporciones_razas.csv",header=True).rdd
proporciones_razas = proporciones_razas.map(lambda x:(x[0],x[1],pd.to_numeric(x[2],"coerce"),\
                        pd.to_numeric(x[3],"coerce"),pd.to_numeric(x[4],"coerce"),pd.to_numeric(x[5],"coerce"),\
                        pd.to_numeric(x[6],"coerce"))).cache()

In [3]:
#muertes: (name, date, race, city, state)
#proporciones_razas: (state, city, share_white, share_black, share_native_american, share_asian, share_hispanic)

In [10]:
#a) Obtener el estado con mayor porcentaje de muertes de personas de raza negra teniendo en cuenta la 
#cantidad total de muertes por parte de oficiales en ese estado. (10 pts)

muertes_por_estado = muertes.\
        map(lambda x:(x[4],(1 if x[2]=="Black" else 0,1)))#(estado,(1 si es negro sino 0,1))
muertes_totales_estado = muertes_por_estado.\
            reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))#(estado,(muertes negras,muertes totales))
muertes_totales_estado.reduce(lambda x,y: x if x[1][0]/x[1][1]>y[1][0]/y[1][1] else y)

('S2', (68, 329))

In [18]:
#b) Obtener los 10 estados con mayor diferencia entre el porcentaje de muertes y el porcentaje de gente 
#de raza negra en ese estado. Para ello, considerar el porcentaje de raza de un estado como el promedio 
#de los valores de sus ciudades.  Por ejemplo si en Texas el porcentaje de muertes de personas de raza 
#negra por parte de la policía es del 36% y el promedio de share_black para Texas es 24% la diferencia es 0.12.

porcentaje_negra_estado = proporciones_razas.\
        map(lambda x:(x[0],(x[3],1))).reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1])).\
        map(lambda x:(x[0],x[1][0]/x[1][1]))#(estado, porcentaje poblacion negra)

muertes_porcentuales = muertes_totales_estado.\
        map(lambda x:(x[0],x[1][0]*100/x[1][1]))#(estado,porcentaje muertes negras)
muertes_porcentuales_y_proporcion_poblacion = muertes_porcentuales.\
        join(porcentaje_negra_estado)#(estado,(muertes porcentuales, porcentaje poblacion))
muertes_porcentuales_y_proporcion_poblacion.takeOrdered(10,lambda x:-__builtin__.abs(x[1][0]-x[1][1]))

[('S3', (17.261904761904763, 67.2)),
 ('S2', (20.66869300911854, 58.4)),
 ('S1', (20.0, 53.35))]